In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
root="/home/sadagopan/amex-default-prediction/"

samples = pd.read_csv(root + "sample_submission.csv")

In [ ]:
nof=4
for i in range(1, 2):
    preds=pd.DataFrame()
    tag = "".join([str((i*3)+j) for j in range(nof)])
    for j in range(3):
        preds = pd.concat([preds, pd.read_csv(root+"test/" + tag + "/x12-" + str((i*nof)+j) + "dtcnt.preds")], ignore_index=True)
    
    print(("x:", preds['Label'].min(), preds['Label'].max()))
    P = preds[preds['target']==1]['Label'].count()
    N = preds[preds['target']==0]['Label'].count()
    TP = preds[(preds['target']==1) & (round(preds['Label']) == 1)]['Label'].count()
    FP = preds[(preds['target']==0) & (round(preds['Label']) == 1)]['Label'].count()
    FN = preds[(preds['target']==1) & (round(preds['Label']) == 0)]['Label'].count()
    TN = preds[(preds['target']==0) & (round(preds['Label']) == 0)]['Label'].count()
    print("TP", TP, "FP", FP, "FN", FN, "TN", TN,
          "total", preds['Label'].count(),
          "Prec:", round((TP / (TP + FP) * 100), 2) ,
         "Acc:",  round(((TP + TN) / (P + N))*100, 2) ,
         "F1:", round((2*TP / (2*TP + FP + FN))*100, 2))

    for j in range(41):
        print("t12-" + str(j) + "dtcnt.csv", end = " ")
#        testdf = (pd.read_csv("t12-" + str(j) + "dtcnt.csv"))
        preds = (pd.read_csv(root+"/test/" + tag + "/t12-" + str(j) + ".dtcnt.preds", names=['customer_ID', 'Label']))
#        preds = predict_model(model, data=testdf, verbose=True)
        preds['Labelnew'] = round(preds['Label'])
        labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
        print(" ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]))
        preds[['customer_ID', 'Label']].to_csv(root+"/test/" + tag + "/t12-"+ str(j) + ".preds", index=False)

In [ ]:
xrange=[]
nof=4

for j in range(5):
    tag = "".join([str((j*nof)+k) for k in range(nof)])
    df=pd.DataFrame()
    for k in range(nof):
        df = pd.concat([df, pd.read_csv(root+"test/" + tag + "/x12-" + str((j*nof)+k)  + ".preds")], ignore_index=True);
    xrange.append((df['Label'].agg(['min', 'max'])[0], df['Label'].agg(['min', 'max'])[1]))
print(xrange)
for i in range(41):
    df = pd.DataFrame()
    for j in range(3):
        tag = "".join([str((j*nof)+k) for k in range(nof)])
        t = pd.read_csv(root+"test/" + tag + "/t12-" + str(i) + ".preds");
        t['xmin'+ tag] = xrange[j][0]
        t['xmax'+ tag] = xrange[j][1]
        if j==0:
            df = t
            df['allxmin'] = df.apply(lambda x: abs(min(x['Label'] - x['xmin'+tag], x['xmax'+tag] - x['Label'])), axis=1)
            df['Label'+tag] = df['Label']
            df['min'] = xrange[j][0]
            df['max'] = xrange[j][1]
        else:
            t.rename(columns={"Label":"Label"+ tag}, inplace=True)
            df = df.merge(t, on="customer_ID", how="inner")
            df['Label'] = df.apply(lambda x: x['Label' + tag] if abs(min(x['Label'+ tag] - x['allxmin'], x['allxmin'] - x['Label'+ tag])) < x['allxmin'] else x['Label'], axis=1)
            df['allxmin'] = df.apply(lambda x: abs(min(x['Label' + tag] - x['allxmin'], x['allxmin'] - x['Label'+ tag])), axis=1)

        df['min'] = df.apply(lambda x: min(x['Label'], x['xmin' + tag]) if x['Label'] == x['Label' + tag] else min(x['Label'], x['min']), axis=1)
        df['max'] = df.apply(lambda x: max(x['Label'], x['xmax' + tag]) if x['Label'] == x['Label' + tag] else max(x['Label'], x['max']), axis=1)
    df['Label'] = ( (df['Label'] - df['min']) / (df['max'] - df['min']) ) * (1 - 0) + 0
    df[['customer_ID', 'Label']].to_csv(root+"/test/" + "t12-"+ str(i) + ".preds", index=False)
    df['Labelnew'] = round(df['Label'])
    labels = df.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
    print("t12-"+ str(i) + ".preds", "total:",labels.sum()[0], " ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]), (labels.loc[1][0]/labels.sum()[0]))
#    print(root+"/test/" + "t12-"+ str(i) + ".preds")


In [65]:
preddf= pd.DataFrame()
rangemin, rangemax = 0, 1
for i in range(12):
    preds = pd.read_csv(root+"/test/x"+ str(i) + "13dtsnew.preds")
    rangemin = preds['Label'].min()
    rangemax = preds['Label'].max()
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
    preds = pd.read_csv(root+"/test/t"+ str(i) + "13dtsnew.preds")
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)
print("")
for i in range(41):
    preds = pd.read_csv(root+"/test/t12-"+ str(i) + ".preds")
    if (preds['Label'].agg(['min', 'max'])[0] < 0) or (preds['Label'].agg(['min', 'max'])[1]>1):
        print(i, preds['Label'].agg(['min', 'max'])[0], preds['Label'].agg(['min', 'max'])[1])
        rangemin = min(preds['Label'].min(), rangemin)
        rangemax = max(preds['Label'].max(), rangemax)
        preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)

print("preds min max:", (preddf['Label'].min(), preddf['Label'].max()))

-0.0413792359063929 1.0677870148510304 actual: -0.0456640665430542 1.0677870148510304
-0.0498692415526941 1.061117875584176 actual: -0.0534139201844086 1.061117875584176
-0.0751450203379568 1.1043908696445988 actual: -0.0751450203379568 1.1180760632835665
-0.0885999557267909 1.1309061107528868 actual: -0.0885999557267909 1.1430827153993284
-0.0735511983060041 1.1174732686105926 actual: -0.0735511983060041 1.131468763135459
-0.0745833463974385 1.106873169201667 actual: -0.0745833463974385 1.106873169201667
-0.0910255496676819 1.1348428897506706 actual: -0.0910255496676819 1.1348428897506706
-0.0952120474528117 1.125113770159423 actual: -0.0952120474528117 1.125113770159423
-0.0952558435989524 1.1020452436169437 actual: -0.0952558435989524 1.1042372586082525
-0.1020667927640645 1.123555135074719 actual: -0.1020667927640645 1.123555135074719
-0.112305928628151 1.0937459194426915 actual: -0.112305928628151 1.1069057105231548
-0.0990895994372982 1.0872057902885186 actual: -0.099089599437298

In [66]:
valids = pd.merge(samples, preddf, on='customer_ID', how='inner')
valids.rename(columns={'prediction': 'target'}, inplace=True)
valids.rename(columns={'Label': 'prediction'}, inplace=True)

valids['prediction'].fillna(0, inplace=True)

print("matching 1's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 1)].count()[0], valids[(valids['target'] == 1)].count()[0])
print("matching 0's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 0)].count()[0], valids[(valids['target'] == 0)].count()[0])
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 1's 0 0
matching 0's 750883 924621
matching 750883 total 924621 % 0.8120981461593453


In [67]:
valids[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)

In [68]:
subs = pd.read_csv("submission.csv")
subs.rename(columns={"prediction":"lnew"}, inplace=True)
subs = pd.merge(subs, pd.read_csv("submission.csv.771"), on="customer_ID", how="inner")
subs.rename(columns={"prediction":"l771"}, inplace=True)
subs = pd.merge(subs, pd.read_csv("submission.csv.770-2"), on="customer_ID", how="inner")
subs.rename(columns={"prediction":"l770-2"}, inplace=True)
subs = pd.merge(subs, pd.read_csv("submission.csv.770"), on="customer_ID", how="inner")
subs.rename(columns={"prediction":"l770"}, inplace=True)
subs = pd.merge(subs, pd.read_csv("submission.csv.765"), on="customer_ID", how="inner")
subs.rename(columns={"prediction":"l765"}, inplace=True)
#subs = pd.merge(subs, pd.read_csv("submission.csv.764"), on="customer_ID", how="inner")
#subs.rename(columns={"prediction":"l764"}, inplace=True)
#subs = pd.merge(subs, pd.read_csv("submission.csv.763-1"), on="customer_ID", how="inner")
#subs.rename(columns={"prediction":"l763-1"}, inplace=True)
#subs = pd.merge(subs, pd.read_csv("submission.csv.763-2"), on="customer_ID", how="inner")
#subs.rename(columns={"prediction":"l763-2"}, inplace=True)
#subs = pd.merge(subs, pd.read_csv("submission.csv.763-3"), on="customer_ID", how="inner")
#subs.rename(columns={"prediction":"l763-3"}, inplace=True)
#subs = pd.merge(subs, pd.read_csv("submission.csv.753"), on="customer_ID", how="inner")
#subs.rename(columns={"prediction":"l753"}, inplace=True)
subs

,customer_ID,lnew,l771,l770-2,l770,l765
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.060515,0.060515,0.060515,0.073283,0.180407
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.080087,0.044230,0.029465,0.029465,0.004871
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.099959,0.097463,0.060005,0.060005,0.042870
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.291007,0.443629,0.318961,0.318961,0.309818
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.507471,0.762211,0.769071,0.769071,0.750628
...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.089497,0.061700,0.042919,0.042919,0.029654
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.680617,0.681420,0.709523,0.709523,0.748113
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.306541,0.502164,0.536819,0.536819,0.695873
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.348837,0.394737,0.319916,0.319916,0.318284


In [58]:
subs.sort_values('lnew', ascending=False).iloc[:36905,:]#[(subs['lnew'] > 0.85) & (subs['l770-3'] < 0.85)]

,customer_ID,lnew,l771,l770-2,l770,l765
288151,4fd192fb4a2728117b1bae3f4bf368ceaeee7dec00c9dd...,1.000000,1.000000,1.000000,1.000000,0.953090
431619,77a454e30db98f18332751f09a2eaa4a8f508018285d91...,1.000000,1.000000,0.978234,0.978234,1.067998
588582,a332e258b3ecee83913311026589da49e39d408d22d12c...,1.000000,1.000000,1.000000,0.997342,0.944024
639379,b130b0f34caaf6e13b4e3a14448b21d00d6c002a368647...,1.000000,1.000000,1.000000,0.989732,0.187982
452301,7d6d30c87c7997fa4e03f3699313228c6dcf7ef486f806...,1.000000,1.000000,0.985934,0.985934,1.075381
...,...,...,...,...,...,...
58648,104d05626fbbbeafb924923faf9b910b63a224b9c9e288...,0.863226,0.863226,0.863226,0.908929,0.839606
102846,1c910310c858ebe86bd3a619c3cbadeb1f1c51549a059a...,0.863222,0.863222,0.863222,0.840805,0.824217
788388,da55e668679409183fb70ae09357c13c538e98964bf35e...,0.863220,0.863220,0.848479,0.848479,0.945243
656926,b6034a87ddd3bdfd5be43d1521b5e35831403e2ab22cc3...,0.863218,0.863218,0.863218,0.817683,0.733298


In [24]:
subs[(subs['l770-3'] > 0.88) & (subs['l770-2'] < 0.88)]

,customer_ID,lnew,l770-3,l770-2,l770,l765,l764,l763-1,l763-2,l763-3,l753
6,00007cfcce97abfa0b4fa0647986157281d01d3ab90de9...,0.930387,0.930387,0.857596,0.857596,0.892400,0.867420,0.852255,0.886746,0.874536,0.867420
58,00041425846d01772af1bd18c7a70535ed88529240540f...,0.880828,0.880828,0.846490,0.846490,0.920505,0.850031,0.814646,0.851167,0.827251,0.850031
80,00059a7fc495b3bc71df7c4d90800d8ee83377ef8fe4ad...,0.883269,0.883269,0.822515,0.822515,0.839233,0.863891,0.801022,0.830777,0.805311,0.863891
113,00087c08673572896c5ddbf6d5867134df277593178f91...,0.890905,0.890905,0.840667,0.840667,0.881719,0.872509,0.802825,0.893734,0.835745,0.872509
117,0008dcf076a0b42b22aa385cae15dc5f84d3e865d43d64...,0.895548,0.895548,0.853501,0.853501,0.889344,0.905779,0.826039,0.842371,0.868880,0.905779
...,...,...,...,...,...,...,...,...,...,...,...
924325,ffeb2a14ddb42abd247ae6946e33821202daa6357e62b2...,0.989587,0.989587,0.867899,0.867899,0.935728,0.876281,0.866947,0.877501,0.875502,0.876281
924370,ffee5adcae9c7e6f3d63a722e23fc50de502abdf2093a4...,0.933443,0.933443,0.714985,0.714985,0.804857,0.810862,0.785869,0.792632,0.731371,0.810862
924427,fff25233e4eb329bd3bc861511f078bb052c40cf2a9796...,0.885121,0.885121,0.840542,0.840542,0.845150,0.803920,0.801699,0.777352,0.776765,0.803920
924468,fff54197d049ece619188fe4daa75497a698668e54db74...,0.895715,0.895715,0.818491,0.818491,0.885476,0.830958,0.838201,0.786156,0.753021,0.830958


In [25]:
subs[(subs['l770-3'] < 0.88) & (subs['l770-2'] > 0.88)]

,customer_ID,lnew,l770-3,l770-2,l770,l765,l764,l763-1,l763-2,l763-3,l753
2803,00c6e932ec1c6e4db8728866248a453c3fd0df236435d1...,0.816666,0.816666,0.881628,0.881628,0.841746,0.808505,0.787752,0.819652,0.789276,0.808505
5322,017ce2e8161392696f1a901d7bd7ea892ebd24d04e512f...,0.875973,0.875973,0.880451,0.880451,0.891330,0.854656,0.793066,0.857623,0.870693,0.854656
6779,01e6303c1ee05623f001d4d71afebf69f7672614e9f103...,0.847343,0.847343,0.900233,0.900233,0.878334,0.861172,0.850097,0.819948,0.861156,0.861172
8668,026c939cb54b41c59e232349884d684d62b1e380b81994...,0.849921,0.849921,0.884179,0.884179,0.884363,0.854066,0.816152,0.831350,0.831645,0.854066
9145,028d42d302bb2464b8d407fbfc43dd51c2dddc56b16bfd...,0.874149,0.874149,0.886750,0.886750,0.919872,0.798837,0.824525,0.825822,0.825448,0.798837
...,...,...,...,...,...,...,...,...,...,...,...
918317,fe4358acfc11a27869ddf08f2dd7fb6c5243e5bcd7797e...,0.844830,0.844830,0.880501,0.880501,0.923592,0.867610,0.810135,0.859561,0.846348,0.867610
919597,fe9e71cd36559a0c283a5e4859830c74017090c9c0971e...,0.872288,0.872288,0.901595,0.901595,0.903324,0.854424,0.808834,0.820020,0.851151,0.854424
920443,fedaf57cd9cecf0fcfc0b66f254003a1a63b0c69e1c674...,0.873731,0.873731,0.922203,0.922203,0.926962,0.916195,0.863305,0.926826,0.891092,0.916195
920890,fef88197b542911721084fe59ae70cd96aa52f616ac883...,0.863613,0.863613,0.902042,0.902042,0.930450,0.827343,0.839136,0.849722,0.845733,0.827343


In [24]:
subs[(subs['lnew']<=0.5)]

,customer_ID,lnew,l770,l765,l764,l763-1,l763-2,l763-3,l753
0,00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.073283,0.073283,0.180407,0.180407,0.180407,0.180407,0.180407,0.135306
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.029465,0.029465,0.004871,0.029465,0.038739,0.038361,0.043173,0.029465
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.060005,0.060005,0.042870,0.060005,0.077655,0.092037,0.060005,0.060005
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.318961,0.318961,0.309818,0.318961,0.372262,0.397523,0.318961,0.318961
5,00004ffe6e01e1b688170bbd108da8351bc4c316eacfef48643ee028ab947afc,0.027020,0.027020,0.003403,0.027020,0.032819,0.032819,0.043381,0.027020
...,...,...,...,...,...,...,...,...,...
924614,ffff76707978792b6eb985027dbe0e85435e90429459c5e1298468b34e302c5a,0.036411,0.036411,0.021217,0.036411,0.060487,0.041552,0.036411,0.036411
924615,ffff824b313399b00db6bc930b83f1a2188d8b1dbd3a31de068bb18b44b1405c,0.028224,0.028224,0.004378,0.028224,0.047523,0.034934,0.044638,0.028224
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.042919,0.042919,0.029654,0.042919,0.078469,0.065474,0.048807,0.042919
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977822e8b356988ca4d07,0.319916,0.319916,0.318284,0.319916,0.273269,0.280532,0.319916,0.319916


In [23]:
pd.set_option("display.max_colwidth", None)